In [259]:
import configparser 
import logging
import datetime as dt
from influxdb import InfluxDBClient, DataFrameClient
import json
import pandas as pd

In [9]:
config = configparser.ConfigParser()
config.read('./configs/monitor.conf')

['./configs/monitor.conf']

In [13]:
def get_influxdb_clients():
    # FIXME
    influxdb_client = InfluxDBClient(
        config.get('INFLUXDB', 'ADDRESS'),
        config.get('INFLUXDB', 'PORT'),
        config.get('INFLUXDB', 'USER'),
        config.get('INFLUXDB', 'PASSWORD'),
        config.get('INFLUXDB', 'DB_NAME'))

    influxdb_dataframe_client = DataFrameClient(
        config.get('INFLUXDB', 'ADDRESS'),
        config.get('INFLUXDB', 'PORT'),
        config.get('INFLUXDB', 'USER'),
        config.get('INFLUXDB', 'PASSWORD'),
        config.get('INFLUXDB', 'DB_NAME'))

    return influxdb_client, influxdb_dataframe_client

In [251]:
uuid = '100316824873160941010780204477983883413'


def get_influxdb_data(client, df_client):

    last_element_query = 'select * from image order by desc limit 1'
    last_result = client.query(last_element_query)
    
    last_uuid = list(last_result.get_points())[0]['uuid']
    
    data_query = 'select * from image where uuid = \'{}\' group by "FILTER"  order by asc'.format(
        last_uuid)
    
#     data_query = 'select * from image group by "FILTER" order by asc limit 100'.format(
#                      uuid
#                     )
    result = df_client.query(data_query)
    result = dict(
        [(k[1][0][1], v.reset_index(level=0).to_json()) for k, v in result.items()])

    return last_result, result


In [252]:
client, df_client = get_influxdb_clients()

In [253]:
last_result, result = get_influxdb_data(client, df_client)

In [254]:
last_result

ResultSet({'('image', None)': [{'time': '2019-05-11T23:45:41.040560128Z', 'BACKGROUND': 1670.906, 'DATE-OBS': None, 'DEC_CENTER': 78.539714, 'DEC_HDR': 78.556122, 'EXPTIME': 10, 'EXPTIME_1': None, 'FILTER': 'B', 'FLUX_MAX': 7463.553, 'FWHM_IMAGE': 3.15, 'NUMBER': 34, 'OBJECT': 'excep', 'OBSERVER': 'MD', 'PHOTOMETRY_STATUS': 1, 'RA_CENTER': 30.649038, 'RA_HDR': 30.997612, 'SNR_WIN': 258.2, 'TIME-OBS': None, 'X_IMAGE': 291.0218, 'Y_IMAGE': 448.347, 'image_time': '2019-03-19T21:00:44.86', 'uuid': '299054492486576389112619756464580679075'}]})

In [255]:
list(last_result.get_points())[0]

{'time': '2019-05-11T23:45:41.040560128Z',
 'BACKGROUND': 1670.906,
 'DATE-OBS': None,
 'DEC_CENTER': 78.539714,
 'DEC_HDR': 78.556122,
 'EXPTIME': 10,
 'EXPTIME_1': None,
 'FILTER': 'B',
 'FLUX_MAX': 7463.553,
 'FWHM_IMAGE': 3.15,
 'NUMBER': 34,
 'OBJECT': 'excep',
 'OBSERVER': 'MD',
 'PHOTOMETRY_STATUS': 1,
 'RA_CENTER': 30.649038,
 'RA_HDR': 30.997612,
 'SNR_WIN': 258.2,
 'TIME-OBS': None,
 'X_IMAGE': 291.0218,
 'Y_IMAGE': 448.347,
 'image_time': '2019-03-19T21:00:44.86',
 'uuid': '299054492486576389112619756464580679075'}

In [276]:
for k, v in result.items():
    a = pd.read_json(v)

In [279]:
a['index']

0     1557618236820
1     1557618242642
10    1557618293402
11    1557618299264
12    1557618305115
13    1557618311153
14    1557618317067
15    1557618322899
16    1557618329022
17    1557618335043
18    1557618341040
2     1557618248637
3     1557618254727
4     1557618260652
5     1557618266570
6     1557618269519
7     1557618275346
8     1557618281348
9     1557618287449
Name: index, dtype: int64